The U.S. Fish and Wildlife Service [7-year Work Plan](https://www.fws.gov/endangered/esa-library/pdf/Listing%207-Year%20Workplan%20Sept%202016.pdf) has been the subject of collaboration between USGS and FWS on the current state of research for these species. As part of this, the Biogeographic Characterization Branch has put together a short review of what our process can say about the current state of data for these species. This is based on the Taxa Information Registry module of the Biogeographic Information System, an intelligent platform we are building to bring together all of our work into a cohesive whole.

This notebook is the result of some previous individual experimentation from members of our staff captured in the GitHub repo where this notebook lives. It runs through all of the processes we've used to assemble a master database for further analysis and reporting.

In [1]:
%env DB_USERNAME=sbristol
%env DB_PASSWORD=h0peTh1sBetter
%env MONGODB_SERVER=54.91.95.139
%env DB_DATABASE=bis

env: DB_USERNAME=sbristol
env: DB_PASSWORD=h0peTh1sBetter
env: MONGODB_SERVER=54.91.95.139
env: DB_DATABASE=bis


In [2]:
from pybis.db import Db as db
from pybis.tess import Tess as tess
from IPython.display import display
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

These are a couple of new functions I created to support the process of getting data from the spreadsheet into our database for further processing. The lookupState function may be useful elsewhere. It uses the "us" Python package to pull together state names and FIPS codes from the state abbreviations in the source data.

In [47]:
def packageESASpeciesRow(row):
    submittedData = {}
    submittedData["Scientific Name"] = row["Scientific Name (Revised List)"]
    submittedData["Species Record Reference"] = row["ScientificNameLink"] 
    submittedData["Common Name"] = row["Species Name (Common)"]
    submittedData["Grouping"] = row["Grouping"]
    submittedData["Lead FWS Region"] = row["Lead FWS Regional Office"]
    submittedData["Species Range"] = row["Species Range"]
    return submittedData

def lookupState(stateAbbr):
    import us
    try:
        return {"name":us.states.mapping('abbr', 'name')[stateAbbr],"fips":us.states.mapping('abbr', 'fips')[stateAbbr]}
    except:
        return None
    
def find_gap_species(criteria):
    from sciencebasepy import SbSession
    sb = SbSession()
    
    base_item_id = "527d0a83e4b0850ea0518326"
    response = sb.find_items({'parentId': base_item_id,'fields':'identifiers','filter':'itemIdentifier={key:'+str(criteria)+'}'})

    if response['total'] == 1:
        gapRecord = {}
        gapRecord['Match Method'] = {'Match Type':next((identifier['type'] for identifier in response['items'][0]['identifiers'] if identifier['key'] == str(criteria)), None),'Criteria':criteria}
        gapRecord['Species Code'] = next((identifier['key'] for identifier in response['items'][0]['identifiers'] if identifier['type'] == 'GAP_SpeciesCode'), None)
        gapRecord['Scientific Name'] = next((identifier['key'] for identifier in response['items'][0]['identifiers'] if identifier['type'] == 'ScientificName'), None)
        gapRecord['Common Name'] = next((identifier['key'] for identifier in response['items'][0]['identifiers'] if identifier['type'] == 'CommonName'), None)
        gapRecord['DOI'] = next((identifier['key'] for identifier in response['items'][0]['identifiers'] if identifier['type'] == 'doi'), None)
        return gapRecord
    else:
        return None


# Import Spreadsheet
The FWS work plan (linked in the intro) has only common names. Staff in the Ecosystems Mission Area put together a spreadsheet that has all of those species but resolves some things to include scientific name and a link to the FWS Ecological Conservation Online system record for the species. This is a really helpful starting point in making sure we have the right taxonomy and links to other systems. To process this, I pulled the links out as a separate column in the spreadsheet and loaded it up into the same folder with this code for processing. We are loading these data into a MongoDB database system we have running on the ESIP Testbed under the DataDistillery project we are continuing to build there.

This code uses Pandas as an expedient way of reading the spreadsheet data from the file into a dataframe.

In [ ]:
df = pd.read_excel("FWS ESA Work Plan Species list for CSS.xlsx", sheet_name='Sheet1')
display (df)

This code uses a private package that we keep secure to connect to the MongoDB instance on the DataDistillery platform and establish a connection to a collection for these data.

In [4]:
bisDB = db.connect_mongodb("bis")
esaWPSpecies = bisDB["FWS_Work_Plan_Species"]

This code runs through the spreadsheet and puts all of the data into an array for submission to the collection.

In [ ]:
fwsESASpeciesList = []

for index,row in df.iterrows():
    speciesRecord = {}
    speciesRecord["Submitted Data"] = packageESASpeciesRow(row)
    speciesRecord["Processing Metadata"] = {"Date Created from Source":datetime.utcnow().isoformat()}
    fwsESASpeciesList.append(speciesRecord)

esaWPSpecies.delete_many({})
esaWPSpecies.insert_many(fwsESASpeciesList)

The FWS work plan includes an indication of species range as a list of states and territories. This seems like an interesting point of comparison with other sources of range information that we might want to analyze, and this code block breaks out the state/territory information into a more robust data structure for later processing and adds in FIPS codes.

In [ ]:
for record in esaWPSpecies.find():
    fwsRange = {"US States":[],"US State List":[],"Other Places":[]}
    for rangePlace in record["Submitted Data"]["Species Range"].replace(", ",",").split(","):
        rangeState = lookupState(rangePlace)
        if rangeState is None:
            fwsRange["Other Places"].append({rangePlace:{}})
        else:
            fwsRange["US States"].append({rangePlace:rangeState})
            fwsRange["US State List"].append(rangePlace)

    if len(fwsRange["Other Places"]) == 0:
        del fwsRange["Other Places"]

    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"FWS Range":fwsRange}})

In [25]:
# Cleanup by putting state abbreviations into a comparable list
for record in esaWPSpecies.find({},{"FWS Range.US State List":1}):
    fwsRange = []
    for stateAbbrev in record["FWS Range"]["US State List"]:
        fwsRange.append(lookupState(stateAbbrev)["name"])
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"Synthesis.FWS Range List":fwsRange}})

In [ ]:
display(esaWPSpecies.find_one())

# ECOS Scrape
The spreadsheet of species on the petition list included links to the FWS Ecological Conservation Online System for most of the species. After working through a few issues trying to connect to the FWS Threatened and Endangered Species (TESS) API for the species, I found it necessary to scrape the ECOS web pages for some additional information. It turns out that the ECOS system has multiple identifiers for species that seem to be used in various parts of the data model and that are not all readily available through their APIs. The public web pages seem to assemble a lot of this information from various places through a back-end app of some kind, but there is no real API that I could find to work against for everything. In order to reliably understand and work with the connections to other systems that FWS folks have put together, it seemed like we needed to go ahead and parse out some information from the human-readable web pages into usable data. This code does that using BeautifulSoup.

In [ ]:
for record in esaWPSpecies.find({"$and":[{"ECOS Scrape":{"$exists":False}},{"Submitted Data.Species Record Reference":{"$not":{"$eq":float("nan")}}}]}):
    ecosScrape = {}
    ecosScrape["url"] = record["Submitted Data"]["Species Record Reference"]
    
    if ecosScrape["url"].find("spcode") > -1:
        ecosScrape["SPCODE"] = ecosScrape["url"].split("=")[1]
    
    ecosContent = requests.get(ecosScrape["url"]).content
    soup = BeautifulSoup(ecosContent,"lxml")

    title = str(soup.find("title").string)
    if title.find("(") == -1:
        ecosScrape["Scientific Name"] = title.replace("Species Profile for ","").strip()
    else:
        ecosScrape["Common Name"] = title.replace("Species Profile for ","").split("(")[0].strip()
        ecosScrape["Scientific Name"] = title.replace("Species Profile for ","").split("(")[1].replace(")","").strip()
        
    
    itisDiv = soup.find("div", {"class": "taxonomy new-row"})
    if itisDiv is not None:
        itisLink = itisDiv.find("a", href=True)
        ecosScrape["TSN"] = itisLink["href"].split("&")[1].split("=")[1]
    
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"ECOS Scrape":ecosScrape}})

In [ ]:
# Show what one of the records looks like at this point with the ECOS scraped information on board.
display(esaWPSpecies.find_one({"ECOS Scrape":{"$exists":True}}))

# TESS
The links for most species to ECOS included in the spreadsheet and mentioned above contains the "SPCODE" identifier from that system. This identifier is different from the SPCODE or ENTITY_ID that is available in other parts of ECOS, and there does not appear to be a public API available to key on that identifier. The web links lead to public landing pages for the species that have a collection of useful information that we may look to parse out for analysis later. For now, we use the species scientific name to find the species in ECOS TESS and bring back any of its information for later use.

The presence of an ITIS TSN identifier assigned to an ECOS species record is a pretty solid identifier to use in retrieving data from the TESS system. This code block uses a TSN type query to retrieve as many records as possible back for the data collection.

In [ ]:
for record in esaWPSpecies.find({"$and":[{"TESS":{"$exists":False}},{"ECOS Scrape.TSN":{"$exists":True}}]}):
    tessData = tess.tess_query(tess.get_tess_search_url("TSN",record["ECOS Scrape"]["TSN"]))
    if tessData["result"] is not False:
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"TESS":tessData}})
    else:
        display(record)

For a few cases, we did not have an ITIS TSN in the data from the ECOS scrape, but we do have an SPCODE identifier in the URL from the link. We can use those to go after TESS data. This code block is meant to run in sequence after trying for TESS data via TSN.

In [5]:
for record in esaWPSpecies.find({"$and":[{"TESS":{"$exists":False}},{"ECOS Scrape.SPCODE":{"$exists":True}}]}):
    tessData = tess.tess_query(tess.get_tess_search_url("SPCODE",record["ECOS Scrape"]["SPCODE"]))
    if tessData["result"] is not False:
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"TESS":tessData}})
    else:
        display(record)

In cases where we still don't have any TESS data after trying ITIS TSN and SPCODE identifiers, we can still try to use the scientific name to see if there is anything in the system. If not, then there must be some reason that FWS has not entered information for a particular petition into their core system.

At this point, I also check to see if the scientific name we scraped from a linked ECOS web page matches the scientific name from the FWS pre-listing plan spreadsheet. If it doesn't match, I put a note in the processing metadata indicating that there is an issue we may want to investigate further. Depending on who established a link to ECOS in the spreadsheet, it may just be that we resolved some taxonomic issue with what was originally submitted by a petitioner.

In [6]:
for record in esaWPSpecies.find({"$and":[{"TESS":{"$exists":False}},{"ECOS Scrape.SPCODE":{"$exists":False}},{"ECOS Scrape.Scientific Name":{"$exists":True}}]}):
    if record["Submitted Data"]["Scientific Name"] != record["ECOS Scrape"]["Scientific Name"]:
        processingMetadata = record["Processing Metadata"]
        processingMetadata["ECOS Match Annotation"] = "Scientific name from spreadsheet didn't match with referenced ECOS record"
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"Processing Metadata":processingMetadata}})

    tessData = tess.tess_query(tess.get_tess_search_url("SCINAME",record["ECOS Scrape"]["Scientific Name"]))
    if tessData["result"] is not False:
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"TESS":tessData}})
    else:
        print("No TESS record found on scientific name search")
        display(record)

At this point, there are a number of records that did not include any ECOS link to follow and scrape and for which we've not been able to retrieve any information from TESS. I go ahead and try to use the original scientific name supplied to run a search with the TESS API to see if we find any results. At this point, we've exhausted all our possibilities of finding a link to TESS without also running ITIS processes to potentially turn up a TSN to search with, so we go ahead and insert a TESS result for every remaining record, indicating that no result was found if that's the case.

In [7]:
for record in esaWPSpecies.find({"TESS":{"$exists":False}}):
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"TESS":tess.tess_query(tess.get_tess_search_url("SCINAME",record["Submitted Data"]["Scientific Name"]))}})


In [8]:
# Show what one of the records looks like at this point with the ECOS scraped information on board.
display(esaWPSpecies.find_one({"TESS.result":True}))

{'ECOS Scrape': {'Common Name': 'Streamside salamander',
  'Scientific Name': 'Ambystoma barbouri',
  'TSN': '208204',
  'url': 'https://ecos.fws.gov/ecp/species/9776'},
 'FWS Range': {'US State List': ['AL', 'KY', 'OH', 'TN', 'WV'],
  'US States': [{'AL': {'fips': '01', 'name': 'Alabama'}},
   {'KY': {'fips': '21', 'name': 'Kentucky'}},
   {'OH': {'fips': '39', 'name': 'Ohio'}},
   {'TN': {'fips': '47', 'name': 'Tennessee'}},
   {'WV': {'fips': '54', 'name': 'West Virginia'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.945491'},
 'Submitted Data': {'Common Name': 'streamside salamander',
  'Grouping': 'Amphibians',
  'Lead FWS Region': 'R4',
  'Scientific Name': 'Ambystoma barbouri',
  'Species Range': 'AL, KY, OH, TN, WV',
  'Species Record Reference': 'https://ecos.fws.gov/ecp/species/9776'},
 'Synthesis': {'FWS Range List': ['Alabama',
   'Kentucky',
   'Ohio',
   'Tennessee',
   'West Virginia']},
 'TESS': {'COMNAME': 'Streamside salamander',
  'CO

# ITIS
One of the interesting data sources that we can tap into is the Integrated Taxonomic Information System, which is managed by part of our group. ITIS is used as the primary taxonomic authority behind the TESS system, and we used previous processes to pull out the ITIS Taxonomic Serial Number (TSN) for many of the species. That is available at this point in both the ECOS Scrape and the TESS data packets.

In [10]:
from pybis.itis import Itis as itis

First, we check just to see if there were any cases where there is a mismatch between the TSNs that we have onboard the items at this point from the ECOS Scrape and the TESS data retrieval processes. If there is an issue, then we need to go back and evaluate the data further to see where there is a misalignment in the FWS data system.

In [11]:
for record in esaWPSpecies.find({"$and":[{"TESS.TSN":{"$exists":True}},{"ECOS Scrape.TSN":{"$exists":True}}]}):
    if record["ECOS Scrape"]["TSN"] != record["TESS"]["TSN"]:
        display (record)

For any of the species records where we could deterimine an ITIS TSN from scraping ECOS landing pages and/or querying TESS, we have to assume that someone did the work to line up with the taxonomic reference. Our first code block here goes through and grabs the base record for the supplied TSN. If that ITIS record shows invalid or unaccepted usage, however, we follow the accepted TSN from the record to also add in the valid/accepted ITIS record.

TESS seems to use some idea of a negative number for ITIS TSN to indicate that a link with ITIS taxonomy has not been established. It's not clear what significance the actual number has, but at this point we skip those and only process records with a TSN greater than 0.

There may be cases where ITIS indicates that the specified record by TSN is invalid or unaccepted. It's not clear what may be happening in these cases. It could be an actual disagreement over taxonomy or something about the information being out of date. In any case, we do go ahead and follow ITIS to retrieve the accepted record, but we also store the record directly linked from the FWS data. Reporting on these cases later may lead to fruitful discussion.

In [12]:
for record in esaWPSpecies.find({"$and":[{"ITIS":{"$exists":False}},{"TESS.TSN":{"$exists":True}}]},{"TESS.TSN":1,"Processing Metadata":1}):
    if int(record["TESS"]["TSN"]) > 0:
        itisByTSN = itis.get_itis_search_url(record["TESS"]["TSN"],False,False)
        try:
            itisResults = requests.get(itisByTSN).json()
        except:
            print ("FAILED QUERY", itisByTSN)
        
        if len(itisResults["response"]["docs"]) == 1:
            itisDocs = [itis.package_itis_json(itisResults["response"]["docs"][0])]
        else:
            display (itisResults)
            
        if "itisDocs" in locals() and itisDocs[0]["usage"] in ["invalid","unaccepted"]:
            if len(itisDocs[0]["acceptedTSN"]) == 1:
                itisByAcceptedTSN = itis.get_itis_search_url(itisDocs[0]["acceptedTSN"][0],False,False)
                itisResults = requests.get(itisByAcceptedTSN).json()
                
                if len(itisResults["response"]["docs"]) == 1:
                    itisDocs.append(itis.package_itis_json(itisResults["response"]["docs"][0]))
                else:
                    display (itisResults)
            else:
                display (itisResults)

    if len(itisDocs) > 0:
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"ITIS":itisDocs}})
    else:
        display (itisResults)


In [13]:
# Show what one of the records with more than one ITIS doc looks like
display(esaWPSpecies.find_one({"ITIS.1":{"$exists":True}}))

{'ECOS Scrape': {'Common Name': 'Arizona toad',
  'Scientific Name': 'Bufo microscaphus microscaphus',
  'TSN': '207135',
  'url': 'https://ecos.fws.gov/ecp/species/2077'},
 'FWS Range': {'US State List': ['AZ', 'CA', 'NM', 'NV', 'UT'],
  'US States': [{'AZ': {'fips': '04', 'name': 'Arizona'}},
   {'CA': {'fips': '06', 'name': 'California'}},
   {'NM': {'fips': '35', 'name': 'New Mexico'}},
   {'NV': {'fips': '32', 'name': 'Nevada'}},
   {'UT': {'fips': '49', 'name': 'Utah'}}]},
 'ITIS': [{'acceptedTSN': ['773525'],
   'cacheDate': '2018-09-07T20:03:00.508258',
   'commonnames': [{'language': 'English', 'name': 'Arizona Toad'}],
   'createDate': '1996-06-13 14:51:08',
   'hierarchy': ['Animalia',
    'Bilateria',
    'Deuterostomia',
    'Chordata',
    'Vertebrata',
    'Gnathostomata',
    'Tetrapoda',
    'Amphibia',
    'Anura',
    'Bufonidae',
    'Anaxyrus',
    'Anaxyrus microscaphus'],
   'kingdom': 'Animalia',
   'nameWInd': 'Bufo microscaphus microscaphus',
   'nameWOInd': '

In [14]:
# What do the records look like that are left over; what do we have to work with in finding possible ITIS matches
for record in esaWPSpecies.find({"ITIS":{"$exists":False}}):
    display (record)

{'FWS Range': {'US State List': ['CT',
   'DE',
   'FL',
   'GA',
   'MA',
   'MD',
   'ME',
   'NC',
   'NH',
   'NJ',
   'NY',
   'PA',
   'RI',
   'SC',
   'VA'],
  'US States': [{'CT': {'fips': '09', 'name': 'Connecticut'}},
   {'DE': {'fips': '10', 'name': 'Delaware'}},
   {'FL': {'fips': '12', 'name': 'Florida'}},
   {'GA': {'fips': '13', 'name': 'Georgia'}},
   {'MA': {'fips': '25', 'name': 'Massachusetts'}},
   {'MD': {'fips': '24', 'name': 'Maryland'}},
   {'ME': {'fips': '23', 'name': 'Maine'}},
   {'NC': {'fips': '37', 'name': 'North Carolina'}},
   {'NH': {'fips': '33', 'name': 'New Hampshire'}},
   {'NJ': {'fips': '34', 'name': 'New Jersey'}},
   {'NY': {'fips': '36', 'name': 'New York'}},
   {'PA': {'fips': '42', 'name': 'Pennsylvania'}},
   {'RI': {'fips': '44', 'name': 'Rhode Island'}},
   {'SC': {'fips': '45', 'name': 'South Carolina'}},
   {'VA': {'fips': '51', 'name': 'Virginia'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.949514'},


{'FWS Range': {'US State List': ['OK'],
  'US States': [{'OK': {'fips': '40', 'name': 'Oklahoma'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.955375'},
 'Submitted Data': {'Common Name': 'Delaware County cave crayfish',
  'Grouping': 'Crustaceans',
  'Lead FWS Region': 'R2',
  'Scientific Name': 'Cambarus subterraneus',
  'Species Range': 'OK',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['Oklahoma']},
 'TESS': {'cacheDate': '2018-09-07T20:00:15.576865', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cc53')}

{'FWS Range': {'US State List': ['TX'],
  'US States': [{'TX': {'fips': '48', 'name': 'Texas'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.956449'},
 'Submitted Data': {'Common Name': 'Texas troglobitic water slater',
  'Grouping': 'Crustaceans',
  'Lead FWS Region': 'R2',
  'Scientific Name': 'Lirceolus smithii',
  'Species Range': 'TX',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['Texas']},
 'TESS': {'cacheDate': '2018-09-07T20:00:16.512714', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cc5b')}

{'FWS Range': {'US State List': ['MO'],
  'US States': [{'MO': {'fips': '29', 'name': 'Missouri'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.957118'},
 'Submitted Data': {'Common Name': 'Big Creek crayfish',
  'Grouping': 'Crustaceans',
  'Lead FWS Region': 'R3',
  'Scientific Name': 'Orconectes peruncus',
  'Species Range': 'MO',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['Missouri']},
 'TESS': {'cacheDate': '2018-09-07T20:00:16.963572', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cc60')}

{'FWS Range': {'US State List': ['MO'],
  'US States': [{'MO': {'fips': '29', 'name': 'Missouri'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.957254'},
 'Submitted Data': {'Common Name': 'St. Francis River crayfish',
  'Grouping': 'Crustaceans',
  'Lead FWS Region': 'R3',
  'Scientific Name': 'Orconectes quadruncus',
  'Species Range': 'MO',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['Missouri']},
 'TESS': {'cacheDate': '2018-09-07T20:00:19.343966', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cc61')}

...

{'FWS Range': {'Other Places': [{'Canada': {}}],
  'US State List': ['NY', 'WI'],
  'US States': [{'NY': {'fips': '36', 'name': 'New York'}},
   {'WI': {'fips': '55', 'name': 'Wisconsin'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.975848'},
 'Submitted Data': {'Common Name': 'bog buck moth',
  'Grouping': 'Insects',
  'Lead FWS Region': 'R5',
  'Scientific Name': 'Hemileuca spp.',
  'Species Range': 'NY, WI, Canada',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['New York', 'Wisconsin']},
 'TESS': {'cacheDate': '2018-09-07T20:00:24.089573', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cce4')}

{'FWS Range': {'US State List': ['MI', 'MN', 'ND', 'WI'],
  'US States': [{'MI': {'fips': '26', 'name': 'Michigan'}},
   {'MN': {'fips': '27', 'name': 'Minnesota'}},
   {'ND': {'fips': '38', 'name': 'North Dakota'}},
   {'WI': {'fips': '55', 'name': 'Wisconsin'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.980341'},
 'Submitted Data': {'Common Name': 'northwestern moose',
  'Grouping': 'Mammals',
  'Lead FWS Region': 'R3',
  'Scientific Name': 'Alces alces andersoni',
  'Species Range': 'MI, MN, ND, WI',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['Michigan',
   'Minnesota',
   'North Dakota',
   'Wisconsin']},
 'TESS': {'cacheDate': '2018-09-07T20:00:24.558045', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cd04')}

{'FWS Range': {'US State List': ['TX'],
  'US States': [{'TX': {'fips': '48', 'name': 'Texas'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.981814'},
 'Submitted Data': {'Common Name': 'Donrichardsonia macroneuron (unnamed moss)',
  'Grouping': 'Non-Flowering Plants',
  'Lead FWS Region': 'R2',
  'Scientific Name': 'Donrichardsonia macroneuron',
  'Species Range': 'TX',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['Texas']},
 'TESS': {'cacheDate': '2018-09-07T20:00:25.003607', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cd0f')}

{'FWS Range': {'US State List': ['NM', 'TX'],
  'US States': [{'NM': {'fips': '35', 'name': 'New Mexico'}},
   {'TX': {'fips': '48', 'name': 'Texas'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.984763'},
 'Submitted Data': {'Common Name': 'Rio Grande cooter',
  'Grouping': 'Reptiles',
  'Lead FWS Region': 'R2',
  'Scientific Name': 'Pseudemys gorzugi',
  'Species Range': 'NM, TX',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['New Mexico', 'Texas']},
 'TESS': {'cacheDate': '2018-09-07T20:00:26.114582', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cd24')}

{'FWS Range': {'US State List': ['CA'],
  'US States': [{'CA': {'fips': '06', 'name': 'California'}}]},
 'Processing Metadata': {'Date Created from Source': '2018-09-07T19:10:01.985940'},
 'Submitted Data': {'Common Name': 'Mojave shoulderband snail',
  'Grouping': 'Snails',
  'Lead FWS Region': 'R8',
  'Scientific Name': 'HELMINTHOGLYPTA GREGGI',
  'Species Range': 'CA',
  'Species Record Reference': nan},
 'Synthesis': {'FWS Range List': ['California']},
 'TESS': {'cacheDate': '2018-09-07T20:00:26.562807', 'result': False},
 '_id': ObjectId('5b92cd0a0bc6be5e9dd2cd2d')}

These all look like species that may not have very complete information in the FWS databases as yet. We didn't find any TESS information for them based on scientific name search. We'll try these against ITIS, but it looks like we may need to do a little bit of name cleanup along the way.

In [17]:
from pybis.bis import Bis as bis
for record in esaWPSpecies.find({"ITIS":{"$exists":False}}):
    itisByName = itis.get_itis_search_url(bis.clean_scientific_name(record["Submitted Data"]["Scientific Name"]),False,False)
    try:
        itisResults = requests.get(itisByName).json()
    except:
        print ("FAILED QUERY", itisByName)

    if len(itisResults["response"]["docs"]) == 1:
        itisDocs = [itis.package_itis_json(itisResults["response"]["docs"][0])]
    else:
        display (itisResults)

    if "itisDocs" in locals() and itisDocs[0]["usage"] in ["invalid","unaccepted"]:
        if len(itisDocs[0]["acceptedTSN"]) == 1:
            itisByAcceptedTSN = itis.get_itis_search_url(itisDocs[0]["acceptedTSN"][0],False,False)
            itisResults = requests.get(itisByAcceptedTSN).json()

            if len(itisResults["response"]["docs"]) == 1:
                itisDocs.append(itis.package_itis_json(itisResults["response"]["docs"][0]))
            else:
                validITISResultDocs = [d for d in itisResults["response"]["docs"] if d["usage"] in ["accepted","valid"]]
                if len(validITISResultDocs) == 1:
                    itisDocs.append(itis.package_itis_json(validITISResultDocs[0]))
                else:
                    display (itisResults)
        else:
            display (itisResults)

    if len(itisDocs) > 0:
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"ITIS":itisDocs}})
    else:
        display (itisResults)


{'response': {'docs': [], 'numFound': 0, 'start': 0},
 'responseHeader': {'QTime': 0,
  'params': {'q': 'nameWOInd:Etheostoma\\ maydeni',
   'rows': '10',
   'wt': 'json'},
  'status': 0}}

{'response': {'docs': [{'_version_': 1610720294635307010,
    'acceptedTSN': ['530889'],
    'createDate': '1996-06-13 14:51:08',
    'credibilityRating': 'TWG standards met',
    'hierarchySoFar': ['530889:$Plantae$Viridiplantae$Streptophyta$Embryophyta$Tracheophyta$Spermatophytina$Magnoliopsida$Lilianae$Asparagales$Asparagaceae$Yucca$Yucca baccata$Yucca baccata var. brevifolia$'],
    'hierarchySoFarWRanks': ['530889:$Kingdom:Plantae$Subkingdom:Viridiplantae$Infrakingdom:Streptophyta$Superdivision:Embryophyta$Division:Tracheophyta$Subdivision:Spermatophytina$Class:Magnoliopsida$Superorder:Lilianae$Order:Asparagales$Family:Asparagaceae$Genus:Yucca$Species:Yucca baccata$Variety:Yucca baccata var. brevifolia$'],
    'hierarchyTSN': ['$202422$954898$846494$954900$846496$846504$18063$846542$897479$810124$43116$43134$530889$'],
    'kingdom': 'Plantae',
    'nameWInd': 'Yucca brevifolia',
    'nameWOInd': 'Yucca brevifolia',
    'otherSource': ['$SRC$7$database$NODC Taxonomic Code$8.0$1996

{'response': {'docs': [], 'numFound': 0, 'start': 0},
 'responseHeader': {'QTime': 0,
  'params': {'q': 'nameWOInd:Donrichardsonia\\ macroneuron',
   'rows': '10',
   'wt': 'json'},
  'status': 0}}

# BISON
One of the other USGS systems we can look to for information is the occurrence data in BISON. In this section, we use ITIS TSN where we've nailed that down for a precise search and scientific name when we don't have an ITIS TSN to retrieve a quick summary of what BISON has to offer. This is based on the code that Abby Benson started but adds a little more to the summary. This summarization is slightly complicated in that some of the originally submitted records may or may not be related to more than one species identifier in BISON.

We are still working on exactly how to go about summarizing BISON data for a given species query, and there are lots of details to work out. For now, I've packaged the same basic logical query that Abby started in an R script here in a Python function. This will have to be tweaked over time and become a new core TIR function at some point as we work out nuances in how the query should operate and how we want to package the data into a more logical format than what is provided by the low level Solr API.

In [19]:
def bison_summary(query):
    import requests
    
    queryBase = "https://data.usgs.gov/solr/occurrences/select/?wt=json&rows=0&facet=true&facet.mincount=1&facet.field=basisOfRecord&facet.field=calculatedState&q="
    queryURL = queryBase+query
    
    bisonSummaryResults = requests.get(queryURL).json()
    
    summaryData = {"query":bisonSummaryResults["responseHeader"]["params"]["q"]}
    summaryData["Total Occurrence Records"] = bisonSummaryResults["response"]["numFound"]

    summaryData["Basis of Record"] = []
    for index,basisOfRecord in enumerate(bisonSummaryResults["facet_counts"]["facet_fields"]["basisOfRecord"]):
        try:
            basisOfRecord += 1
        except:
            summaryData["Basis of Record"].append({basisOfRecord:bisonSummaryResults["facet_counts"]["facet_fields"]["basisOfRecord"][index+1]})

            
    summaryData["US State Occurrences"] = []
    for index,calculatedState in enumerate(bisonSummaryResults["facet_counts"]["facet_fields"]["calculatedState"]):
        try:
            calculatedState += 1
        except:
            if len(calculatedState) == 0:
                calculatedStateValue = "Unknown"
            else:
                calculatedStateValue = calculatedState
            summaryData["US State Occurrences"].append({calculatedStateValue:bisonSummaryResults["facet_counts"]["facet_fields"]["calculatedState"][index+1]})

    return summaryData


We are still working out what to use in querying BISON. We could use some combination of names, but the BISON API is supposed to be intelligent enough to leverage ITIS in the background to return records for synonyms, homonyms, etc. For now, I simply wrote this to try and use the original submitted scientific name to see what we turn up and start fiddling with how to use the summary information.

The potentially interesting thing to examine here at a very crude level of sophistication is the difference between stated range of a species from the standpoint of the FWS review process or the original petition for listing and the number of states where there appear to be occurrence records. Further examination of the BISON data would be needed to determine whether some of the occurrences should be filtered out based on basis of record (e.g., maybe we don't want to consider fossil records), age of the record, lack of completeness, spatial data quality concerns, or other factors.

In [20]:
for record in esaWPSpecies.find({"BISON":{"$exists":False}}):
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"BISON":bison_summary("scientificName:"+record["Submitted Data"]["Scientific Name"])}})

For later comparison with other lists of states, we send a simple list of any state in the BISON data with an occurrence record to a list in the synthesis structure.

In [24]:
import us
for record in esaWPSpecies.find({"BISON":{"$exists":True}},{"BISON.US State Occurrences":1}):
    bisonMasterList = list(set().union(*(d.keys() for d in record["BISON"]["US State Occurrences"])))
    bisonStateList = []
    for state in bisonMasterList:
        if us.states.lookup(state) is not None:
            bisonStateList.append(state)
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"Synthesis.States with BISON Occurrence Data":bisonStateList}})

In [22]:
# Show what one of the records with a BISON summary looks like
display(esaWPSpecies.find_one({"BISON":{"$exists":True}}))

{'BISON': {'Basis of Record': [{'specimen': 5564},
   {'fossil': 1986},
   {'unknown': 66},
   {'observation': 16}],
  'Total Occurrence Records': 7632,
  'US State Occurrences': [{'Michigan': 1156},
   {'Ohio': 709},
   {'New York': 575},
   {'Connecticut': 337},
   {'California': 306},
   {'Ontario Canada': 184},
   {'New Jersey': 155},
   {'Vermont': 154},
   {'Massachusetts': 121},
   {'Kentucky': 78},
   {'Maine': 67},
   {'Indiana': 63},
   {'Florida': 56},
   {'Illinois': 51},
   {'Pennsylvania': 48},
   {'Tennessee': 45},
   {'Arkansas': 36},
   {'Texas': 26},
   {'Virginia': 26},
   {'New Hampshire': 23},
   {'Missouri': 18},
   {'Wisconsin': 16},
   {'Nebraska': 15},
   {'Idaho': 12},
   {'South Carolina': 12},
   {'Kansas': 11},
   {'Quebec Canada': 11},
   {'Mississippi': 8},
   {'Colorado': 7},
   {'Louisiana': 5},
   {'Alabama': 4},
   {'Maryland': 4},
   {'North Carolina': 4},
   {'Washington': 4},
   {'West Virginia': 4},
   {'British Columbia Canada': 3},
   {'Georgia'

# Name Check
At this point, we have started with a submitted scientific name and then pulled together several other potential scientific names for the species.

* Name scraped from referenced ECOS web pages
* Name accessed from the TESS system API
* One or more names from ITIS

Before advancing further, I put unique names by which a species may be known together into a list so that we can look for cases where there is some disagreement and need for future analysis or annotation.

In [23]:
for record in esaWPSpecies.find():
    synthesis = {"Scientific Names":[{"Scientific Name":" ".join(record["Submitted Data"]["Scientific Name"].split()),"Source":"Submitted Data"}]}

    if "ECOS Scrape" in record.keys():
        synthesis["Scientific Names"].append({"Scientific Name":" ".join(record["ECOS Scrape"]["Scientific Name"].split()),"Source":"ECOS Scrape"})

    if "SCINAME" in record["TESS"].keys():
        synthesis["Scientific Names"].append({"Scientific Name":" ".join(record["TESS"]["SCINAME"].split()),"Source":"TESS"})

    for itisDoc in record["ITIS"]:
        synthesis["Scientific Names"].append({"Scientific Name":" ".join(itisDoc["nameWInd"].split()),"Source":"ITIS","Usage":itisDoc["usage"]})

    synthesis["Unique Scientific Names"] = list(set([d["Scientific Name"] for d in synthesis["Scientific Names"]]))
    
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{'Synthesis.Unique Scientific Names':synthesis['Unique Scientific Names'],'Synthesis.Scientific Names':synthesis['Scientific Names']}})    
    

# SGCN
One of the other data systems we maintain is a synthesis of State Species of Greatest Conservation Need. Building on work that Abby contributed, I've pulled the same information together here in a slightly different format to be consistent with other "data packets" we are assembling.

In [26]:
sgcnSynthesis = bisDB["SGCN Synthesis"]

Deviating a little bit from what Abby started, where she did not have an ITIS TSN to go on, this first process works through the work plan records, grabs a valid ITIS TSN, and uses that to search against the SGCN Synthesis (which also happens to run in the same database infrastructure we are building on here). This should give us a good first cut, and then we'll have to rely on other methods to find additional matches to SGCN data.

In [27]:
for record in esaWPSpecies.find({"SGCN":{"$exists":False}}):
    l_validITIStsn = [i for i in record["ITIS"] if i["usage"] in ["valid","accepted"]]
    
    if len(l_validITIStsn) == 1:
        validITIStsn = l_validITIStsn[0]["tsn"]
    
        sgcnSynthesisRecord = sgcnSynthesis.find_one({"ITIS":{"$elemMatch":{"tsn":validITIStsn}}})

        if sgcnSynthesisRecord is not None:
            sgcnSummary = {"Scientific Name":sgcnSynthesisRecord["_id"]}
            sgcnSummary["Common Name"] = sgcnSynthesisRecord["Common Name"]
            sgcnSummary["Match Method"] = sgcnSynthesisRecord["Match Method"]
            sgcnSummary["Taxonomic Authority ID"] = sgcnSynthesisRecord["Taxonomic Authority ID"]
            sgcnSummary["Taxonomy"] = sgcnSynthesisRecord["Taxonomy"]
            sgcnSummary["State List Summary"] = sgcnSynthesisRecord["Source Data Summary"]

            esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"SGCN":sgcnSummary}})


For the leftovers, this code block tries to find a match on the originally submitted scientific name with the SGCN Synthesis.

In [28]:
for record in esaWPSpecies.find({"SGCN":{"$exists":False}}):
    sgcnSynthesisRecord = sgcnSynthesis.find_one({"_id":record["Submitted Data"]["Scientific Name"]})

    if sgcnSynthesisRecord is not None:
        sgcnSummary = {"Scientific Name":sgcnSynthesisRecord["_id"]}
        sgcnSummary["Common Name"] = sgcnSynthesisRecord["Common Name"]
        sgcnSummary["Match Method"] = sgcnSynthesisRecord["Match Method"]
        sgcnSummary["Taxonomic Authority ID"] = sgcnSynthesisRecord["Taxonomic Authority ID"]
        sgcnSummary["Taxonomy"] = sgcnSynthesisRecord["Taxonomy"]
        sgcnSummary["State List Summary"] = sgcnSynthesisRecord["Source Data Summary"]
        
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"SGCN":sgcnSummary}})


In [30]:
# Show what one of the records with an SGCN summary looks like
display(esaWPSpecies.find_one({"SGCN":{"$exists":True}},{"SGCN":1}))

{'SGCN': {'Common Name': 'Streamside Salamander',
  'Match Method': 'Exact Match',
  'Scientific Name': 'Ambystoma barbouri',
  'State List Summary': [{'Common Name': 'Streamside Salamander ',
    'Scientific Name': 'Ambystoma barbouri ',
    'State': 'Ohio',
    'Year': '2015'},
   {'Common Name': 'Streamside Salamander',
    'Scientific Name': 'Ambystoma barbouri',
    'State': 'Tennessee',
    'Year': '2015'},
   {'Common Name': 'Streamside Salamander',
    'Scientific Name': 'Ambystoma barbouri',
    'State': 'West Virginia',
    'Year': '2015'},
   {'Common Name': 'Streamside Salamander',
    'Scientific Name': 'Ambystoma barbouri',
    'State': 'Ohio',
    'Year': '2005'},
   {'Common Name': 'Streamside Salamander',
    'Scientific Name': 'Ambystoma barbouri',
    'State': 'West Virginia',
    'Year': '2005'},
   {'Common Name': 'Streamside Salamander',
    'Scientific Name': 'Ambystoma barbouri',
    'State': 'Kentucky',
    'Year': '2015'},
   {'Common Name': 'Streamside Salama

Add synthesis attributes for the list of states listing each species in their SWAP in 2005 and 2015.

In [34]:
for record in esaWPSpecies.find({"SGCN":{"$exists":True}},{"SGCN":1}):
    esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"Synthesis.SGCN State List 2005":[stateRecord['State'] for stateRecord in record['SGCN']['State List Summary'] if stateRecord['Year'] == '2005'],"Synthesis.SGCN State List 2015":[stateRecord['State'] for stateRecord in record['SGCN']['State List Summary'] if stateRecord['Year'] == '2015']}})

# NatureServe
The NatureServe species data system provides another potentially useful suite of information for us to assemble and work against. We've used it in other applications of the Taxa Information Registry and have some functions that handle lookup and secure, authorized retrieval of information.

In [36]:
from pybis.natureserve import Natureserve as natureserve

Because we previously put together a synthesis of the possible scientific names for a record and laid out a unique names list, we can now work through those unique names looking for matches from the NatureServe system. Because this list is an ordered list, starting with the originally submitted name, that order suits us here. We will break off searching as soon as we find a viable match and return an Element Global ID for further exploration.

In [43]:
for record in esaWPSpecies.find({"NatureServe":{"$exists":False}},{"Submitted Data.Scientific Name":1}):
    nsData = natureserve.query_natureserve(record['Submitted Data']['Scientific Name'])
    if nsData is not None:
        esaWPSpecies.update_one({"_id":record["_id"]},{"$set":{"NatureServe":nsData}})


KeyboardInterrupt: 

In [42]:
# Show what one of the records looks like with NatureServe data onboard
display(esaWPSpecies.find_one({"NatureServe":{"$exists":True}},{"NatureServe":1}))

{'NatureServe': {'@type': 'Animal',
  '@uid': 'ELEMENT_NATIONAL.2.162738',
  'jurisdictionNationName': {'#text': 'UNITED STATES', '@code': 'US'},
  'metadata': {'references': None},
  'nationalCommonName': 'Streamside Salamander',
  'nationalConservationStatus': {'#text': 'N4',
   '@lastChangedDate': '1996-11-05',
   '@lastReviewedDate': '1996-11-05'},
  'nationalDistributions': {'nationalDistribution': {'currentPresenceAbsence': 'Present',
    'distributionConfidence': 'Confident',
    'origin': 'Native',
    'population': 'Year-round',
    'regularity': 'Regularly occurring'}},
  'nationalScientificName': {'formattedName': '<i>Ambystoma barbouri</i>',
   'nomenclaturalAuthor': 'Kraus and Petranka, 1989',
   'unformattedName': 'Ambystoma barbouri'},
  'natureServeGlobalConcept': {'@uid': 'ELEMENT_GLOBAL.2.100100',
   'classificationStatus': 'Standard',
   'globalConservationStatus': 'G4',
   'globalScientificName': {'formattedName': '<i>Ambystoma barbouri</i>',
    'nomenclaturalAutho

# GAP Species
The Gap Analysis Project is in the process of releasing a full set of 1,719 habitat distribution maps for the terrestrial vertebrates with range in the continguous US. We are also working on a set of fish distribution models that follow a somewhat different methodology. As these products evolve, we will include methods that summarize this information for use with the work plan species.

In [58]:
foundByOverrideList = 0
foundByTSN = 0
foundByScientificName = 0
foundByCommonName = 0
notFound = 0

overrideList = []
overrideList.append({"Common Name":"Texas salamander","GAP Species Code":"aTXSAx"})
overrideList.append({"Common Name":"alligator snapping turtle","GAP Species Code":"rASTUx"})
overrideList.append({"Common Name":"Arizona striped whiptail","GAP Species Code":"rASWHx"})
overrideList.append({"Common Name":"Florida sandhill crane","GAP Species Code":"bSACRx"})
overrideList.append({"Common Name":"Comal Springs salamander","GAP Species Code":""})

for record in esaWPSpecies.find({"GAP":{"$exists":False}},{"Submitted Data":1,"ECOS Scrape":1}):
    if record["Submitted Data"]["Common Name"] in [sp["Common Name"] for sp in overrideList]:
        correctGapSpeciesCode = next((sp["GAP Species Code"] for sp in overrideList if sp["Common Name"] == record["Submitted Data"]["Common Name"]), None)
        if len(correctGapSpeciesCode) > 0:
            gapRecord = find_gap_species(correctGapSpeciesCode)
            if gapRecord is not None:
                foundByOverrideList = foundByOverrideList + 1
    else:
        if 'ECOS Scrape' in record.keys() and 'TSN' in record['ECOS Scrape'].keys() and record['ECOS Scrape']['TSN'] is not None:
            gapRecord = find_gap_species(record['ECOS Scrape']['TSN'])
            if gapRecord is not None:
                foundByTSN = foundByTSN + 1
        else:
            gapRecord = find_gap_species(record['Submitted Data']['Scientific Name'])
            if gapRecord is not None:
                foundByScientificName = foundByScientificName + 1
            else:
                try:
                    gapRecord = find_gap_species(record['Submitted Data']['Common Name'])
                    if gapRecord is not None:
                        foundByCommonName= foundByCommonName + 1
                except:
                    continue

    if gapRecord is None:
        notFound = notFound + 1
    else:
        esaWPSpecies.update_one({'_id':record['_id']},{'$set':{'GAP.Species':gapRecord}})

        
print("Override Matches", foundByOverrideList)
print("TSN Matches", foundByTSN)
print("Scientific Name Matches", foundByScientificName)
print("Common Name Matches", foundByCommonName)
print("Not Found", notFound)

Override Matches 4
TSN Matches 58
Scientific Name Matches 3
Common Name Matches 0
Not Found 296


## GAP Species Metrics

Once we've nailed down connections to GAP species, there are a number of interesting things we can start doing with the habitat map data. In our current infrastructure, we've pre-calculated a set of metrics from the habitat maps and protected areas data to provide summary stats for various geographic areas, including CONUS states.

In the following codeblock, I use a still beta API we are preparing for a new version of the National Biogeographic Map to pull back summary stats by state for the GAP species matches and then further synthesize a simple state list for comparison. I'm doing that this way because we hope to be spinning up a whole new process behind all the GAP data such that we can run these kinds of analyses live using any arbitrary polygon of interest, returning additional variables from the data, rather than relying on precalculated metrics. When we get there, we'll update the API to work basically the same way but against live data processing. In order to hit this API at the moment, you need to be connected to the DOI network as it's on a staging deployment on internally accessible cloud resources. You'll see a bunch of SSL warnings here because the HTTPS connection is against something with a self-generated SSL cert.

In [60]:
for record in esaWPSpecies.find({"$and":[{"GAP.Species":{"$exists":True}},{"GAP.State Metrics":{"$exists":False}}]},{"GAP":1}):
    esResults = requests.get("https://master.staging.sciencebase.gov/bis/api/v0.1/elastic/search/bis__bis__state_species_prot_summ?q=%7B%22size%22:48,%22query%22%3A%7B%22match%22%3A%7B%22properties.sppcode%22%3A%22"+record["GAP"]["Species"]["Species Code"]+"%22%7D%7D%7D", verify=False).json();
    gapAnalysisByState = []
    for hit in esResults['hits']['hits']:
        gapAnalysisByState.append(hit['_source']['properties'])
    esaWPSpecies.update_one({'_id':record['_id']},{'$set':{'GAP.State Metrics':gapAnalysisByState}})

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is be

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is be

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is be

In [63]:
for record in esaWPSpecies.find({"GAP.State Metrics":{"$exists":True}},{"GAP.State Metrics":1}):
    esaWPSpecies.update_one({'_id':record['_id']},{'$set':{'Synthesis.States with GAP Species Potential Habitat':[sm['state_name'] for sm in record['GAP']['State Metrics'] if sm['state_name'] != 'CONUS']}})

In [65]:
display(esaWPSpecies.find_one({'GAP':{'$exists':True}},{'GAP':1,'Synthesis':1}))

{'GAP': {'Species': {'Common Name': 'Streamside Salamander',
   'DOI': 'doi:10.5066/F74J0CCV',
   'Match Method': {'Criteria': '208204', 'Match Type': 'itis_tsn_validMatch'},
   'Scientific Name': 'Ambystoma barbouri',
   'Species Code': 'aSTRSx'},
  'State Metrics': [{'gapstat123ac': 54237.654286452,
    'gapstat123group': '1-10',
    'gapstat123perc': 6.95001732659724,
    'gapstat12ac': 18104.2745811174,
    'gapstat12group': '1-10',
    'gapstat12perc': 2.31988318225756,
    'gapstat1ac': 4189.2516557073,
    'gapstat1perc': 0.53681103977822,
    'gapstat2ac': 13915.0229254101,
    'gapstat2perc': 1.78307214247934,
    'gapstat3ac': 36133.3797053346,
    'gapstat3perc': 4.63013414433967,
    'gapstat4ac': 726158.3035608504,
    'gapstat4perc': 93.0499826734028,
    'id': 16595,
    'spp_comname': 'Streamside Salamander',
    'spp_sciname': 'Ambystoma barbouri',
    'sppcode': 'aSTRSx',
    'state_fipscode': '39',
    'state_name': 'Ohio',
    'taxa': 'A',
    'totalac': 780395.9578